##### Respond in a format

In this example, we will build a chat executor that responds in a specific format. We will do this by using OpenAI Function calling, this is useful when you wnat to enforce the agent's response to be in a specific format. In this example, we will ask if it repsond as if a weatherman, so to return the temperature and then any other additonal information

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
tools = [TavilySearchResults(max_results=1)]

In [4]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")


In [6]:
from langchain.tools.render import format_tool_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

class Response(BaseModel):
    """ 
    Final response to the user
    """
    temperature: float = Field(description="the temperature")
    other_notes: str = Field(description="any other notes about the weather")

In [7]:
functions = [format_tool_to_openai_function(tool) for tool in tools]
functions.append(convert_pydantic_to_openai_function(Response))
model = model.bind_functions(functions)

In [8]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage, FunctionMessage, HumanMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [9]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

In [10]:
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    elif last_message.additional_kwargs["tool_calls"][0]["function"]["name"] == "Response":
        return "end"
    else: 
        return "continue"

In [11]:
def call_model(state: AgentState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


def call_tool(state: AgentState):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    fn = last_message.additional_kwargs["tool_calls"][0]["function"]
    
    action = ToolInvocation(
        tool=fn["name"],
        tool_input=json.loads(fn["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

In [12]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

In [13]:
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

workflow.set_entry_point("agent")

workflow.add_conditional_edges("agent", should_continue, {
        "continue": "action",
        "end": END
    })

workflow.add_edge("action", "agent")

app = workflow.compile()

In [17]:
inputs = {"messages": [HumanMessage(content="what is the weather in SF? check using the search tool, finally use the response tool")]}

for output in app.stream(inputs):
    for key, value in output.items():
        print(f"output from node {key}")
        print("----")
        print(value)
    print("----")

output from node agent
----
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_apzundn8m02fjx62qro0w6we', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]})]}
----
output from node action
----
{'messages': [FunctionMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "Weather in San Francisco is {\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1708048121, \'localtime\': \'2024-02-15 17:48\'}, \'current\': {\'last_updated_epoch\': 1708047900, \'last_updated\': \'2024-02-15 17:45\', \'temp_c\': 13.9, \'temp_f\': 57.0, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 11.9, \'wind_kph\': 19.1, \'wind_degr